In [18]:
import pandas as pd
import numpy as np
import random
from collections import deque

class CustomEnv:
    # A custom Bitcoin trading environment
    def __init__(self, df, initial_balance=1000, lookback_window_size=50):
        # Define action space and state size and other custom parameters
        self.df = df.dropna().reset_index()
        self.df_total_steps = len(self.df)-1
        self.initial_balance = initial_balance
        self.lookback_window_size = lookback_window_size

        # Action space from 0 to 3, 0 is hold, 1 is buy, 2 is sell
        self.action_space = np.array([0, 1, 2])

        # Orders history contains the balance, net_worth, crypto_bought, crypto_sold, crypto_held values for the last lookback_window_size steps
        self.orders_history = deque(maxlen=self.lookback_window_size)
        
        # Market history contains the OHCL values for the last lookback_window_size prices
        self.market_history = deque(maxlen=self.lookback_window_size)

        # State size contains Market+Orders history for the last lookback_window_size steps
        self.state_size = (self.lookback_window_size, 10)
    # Reset the state of the environment to an initial state
    def reset(self, env_steps_size = 0):
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.prev_net_worth = self.initial_balance
        self.crypto_held = 0
        self.crypto_sold = 0
        self.crypto_bought = 0
        if env_steps_size > 0: # used for training dataset
            self.start_step = random.randint(self.lookback_window_size, self.df_total_steps - env_steps_size)
            self.end_step = self.start_step + env_steps_size
        else: # used for testing dataset
            self.start_step = self.lookback_window_size
            self.end_step = self.df_total_steps

        self.current_step = self.start_step

        for i in reversed(range(self.lookback_window_size)):
            current_step = self.current_step - i
            self.orders_history.append([self.balance, self.net_worth, self.crypto_bought, self.crypto_sold, self.crypto_held])
            self.market_history.append([self.df.loc[current_step, 'open'],
                                    self.df.loc[current_step, 'high'],
                                    self.df.loc[current_step, 'low'],
                                    self.df.loc[current_step, 'close'],
                                    self.df.loc[current_step, 'volume']
                                    ])

        state = np.concatenate((self.market_history, self.orders_history), axis=1)
        return state
    
    def _next_observation(self):
        self.market_history.append([self.df.loc[self.current_step, 'open'],
                                    self.df.loc[self.current_step, 'high'],
                                    self.df.loc[self.current_step, 'low'],
                                    self.df.loc[self.current_step, 'close'],
                                    self.df.loc[self.current_step, 'volume']
                                    ])
        obs = np.concatenate((self.market_history, self.orders_history), axis=1)
        return obs
    
    def step(self, action):
        self.crypto_bought = 0
        self.crypto_sold = 0
        self.current_step += 1

        # Set the current price to a random price between open and close
        current_price = random.uniform(
            self.df.loc[self.current_step, 'open'],
            self.df.loc[self.current_step, 'close'])

        if action == 0: # Hold
            pass

        elif action == 1 and self.balance > 0:
            # Buy with 100% of current balance
            self.crypto_bought = self.balance / current_price
            self.balance -= self.crypto_bought * current_price
            self.crypto_held += self.crypto_bought

        elif action == 2 and self.crypto_held>0:
            # Sell 100% of current crypto held
            self.crypto_sold = self.crypto_held
            self.balance += self.crypto_sold * current_price
            self.crypto_held -= self.crypto_sold

        self.prev_net_worth = self.net_worth
        self.net_worth = self.balance + self.crypto_held * current_price

        self.orders_history.append([self.balance, self.net_worth, self.crypto_bought, self.crypto_sold, self.crypto_held])

        # Calculate reward
        reward = self.net_worth - self.prev_net_worth

        if self.net_worth <= self.initial_balance/2:
            done = True
        else:
            done = False

        obs = self._next_observation()

        return obs, reward, done

    def render(self):
        print(f'Step: {self.current_step}, Net Worth: {self.net_worth}')

In [20]:
import os

def Write_to_file(Date, net_worth, filename='{}.txt'.format(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))):
    for i in net_worth: 
        Date += " {}".format(i)
    #print(Date)
    if not os.path.exists('logs'):
        os.makedirs('logs')
    file = open("logs/"+filename, 'a+')
    file.write(Date+"\n")
    file.close()

NameError: name 'datetime' is not defined

In [12]:
from tvDatafeed import TvDatafeed, Interval

username = 'No_Sock'
password = '#TradeCore$23'

tv = TvDatafeed(username, password)

In [13]:
cdpr_data = tv.get_hist(symbol='CDR',exchange='GPW',interval=Interval.in_1_hour,n_bars=10000)

In [14]:
def Random_games(env, train_episodes = 50, training_batch_size=500):
    average_net_worth = 0
    for episode in range(train_episodes):
        state = env.reset(env_steps_size = training_batch_size)

        while True:
            env.render()

            action = np.random.randint(3, size=1)[0]

            state, reward, done = env.step(action)

            if env.current_step == env.end_step:
                average_net_worth += env.net_worth
                print("net_worth:", env.net_worth)
                break

    print("average_net_worth:", average_net_worth/train_episodes)

In [19]:
df = cdpr_data[['open','high','low','close','volume']]
lookback_window_size = 50
train_df = df[:-720-lookback_window_size]
test_df = df[-720-lookback_window_size:] # 30 days

train_env = CustomEnv(train_df, lookback_window_size=lookback_window_size)
test_env = CustomEnv(test_df, lookback_window_size=lookback_window_size)

Random_games(train_env, train_episodes = 10, training_batch_size=500)

Step: 840, Net Worth: 1000
Step: 841, Net Worth: 1000.0
Step: 842, Net Worth: 1000.0
Step: 843, Net Worth: 1000.0
Step: 844, Net Worth: 1000.0
Step: 845, Net Worth: 1000.0
Step: 846, Net Worth: 1000.0
Step: 847, Net Worth: 1000.0
Step: 848, Net Worth: 1000.0
Step: 849, Net Worth: 1009.556999129296
Step: 850, Net Worth: 1019.5638856458929
Step: 851, Net Worth: 1019.5638856458929
Step: 852, Net Worth: 1017.6314773407438
Step: 853, Net Worth: 1017.6314773407438
Step: 854, Net Worth: 1023.0578255490369
Step: 855, Net Worth: 1022.9058816846754
Step: 856, Net Worth: 1013.6677159938861
Step: 857, Net Worth: 1013.6677159938861
Step: 858, Net Worth: 1013.6677159938861
Step: 859, Net Worth: 1009.8224607400283
Step: 860, Net Worth: 1009.8224607400283
Step: 861, Net Worth: 1009.8224607400283
Step: 862, Net Worth: 1009.8224607400283
Step: 863, Net Worth: 1005.1106771359372
Step: 864, Net Worth: 995.0984992040351
Step: 865, Net Worth: 985.7694951930931
Step: 866, Net Worth: 987.9375008772992
Step: 8

Step: 1264, Net Worth: 1034.631202321945
Step: 1265, Net Worth: 1034.631202321945
Step: 1266, Net Worth: 1032.0797944828337
Step: 1267, Net Worth: 1032.0797944828337
Step: 1268, Net Worth: 1030.1907698814653
Step: 1269, Net Worth: 1030.3311682331514
Step: 1270, Net Worth: 1030.3311682331514
Step: 1271, Net Worth: 1030.3311682331514
Step: 1272, Net Worth: 1030.3311682331514
Step: 1273, Net Worth: 1031.0617116987191
Step: 1274, Net Worth: 1029.1934154633166
Step: 1275, Net Worth: 1027.064591130184
Step: 1276, Net Worth: 1028.002927084063
Step: 1277, Net Worth: 1028.002927084063
Step: 1278, Net Worth: 1027.6872157938012
Step: 1279, Net Worth: 1027.6872157938012
Step: 1280, Net Worth: 1027.6872157938012
Step: 1281, Net Worth: 1031.435989062004
Step: 1282, Net Worth: 1031.435989062004
Step: 1283, Net Worth: 1029.39699259033
Step: 1284, Net Worth: 1031.6069898721462
Step: 1285, Net Worth: 1051.4829246755214
Step: 1286, Net Worth: 1056.0255052360824
Step: 1287, Net Worth: 1056.0255052360824
S

Step: 3384, Net Worth: 965.3294249314873
Step: 3385, Net Worth: 965.3294249314873
Step: 3386, Net Worth: 965.3294249314873
Step: 3387, Net Worth: 960.568768815828
Step: 3388, Net Worth: 960.568768815828
Step: 3389, Net Worth: 960.568768815828
Step: 3390, Net Worth: 960.568768815828
Step: 3391, Net Worth: 960.568768815828
Step: 3392, Net Worth: 960.568768815828
Step: 3393, Net Worth: 960.568768815828
Step: 3394, Net Worth: 960.568768815828
Step: 3395, Net Worth: 960.568768815828
Step: 3396, Net Worth: 960.568768815828
Step: 3397, Net Worth: 960.568768815828
Step: 3398, Net Worth: 960.568768815828
Step: 3399, Net Worth: 960.568768815828
Step: 3400, Net Worth: 960.568768815828
Step: 3401, Net Worth: 960.568768815828
Step: 3402, Net Worth: 960.568768815828
Step: 3403, Net Worth: 984.9671305955479
Step: 3404, Net Worth: 986.1312274559755
Step: 3405, Net Worth: 1001.9158687906088
Step: 3406, Net Worth: 1009.5224850938575
Step: 3407, Net Worth: 1009.5224850938575
Step: 3408, Net Worth: 1009.5

Step: 3498, Net Worth: 1021.4975009325855
Step: 3499, Net Worth: 1018.4998136052025
Step: 3500, Net Worth: 1045.36359313801
Step: 3501, Net Worth: 1038.048576043978
Step: 3502, Net Worth: 1032.340707819481
Step: 3503, Net Worth: 1032.340707819481
Step: 3504, Net Worth: 1032.340707819481
Step: 3505, Net Worth: 1032.340707819481
Step: 3506, Net Worth: 1032.340707819481
Step: 3507, Net Worth: 1032.340707819481
Step: 3508, Net Worth: 1032.340707819481
Step: 3509, Net Worth: 1021.7110379572426
Step: 3510, Net Worth: 1028.3425497208657
Step: 3511, Net Worth: 1032.6501246424098
Step: 3512, Net Worth: 1062.5144569388124
Step: 3513, Net Worth: 1064.4492291191907
Step: 3514, Net Worth: 1072.685357611362
Step: 3515, Net Worth: 1072.685357611362
Step: 3516, Net Worth: 1072.685357611362
Step: 3517, Net Worth: 1081.5969878247506
Step: 3518, Net Worth: 1081.5969878247506
Step: 3519, Net Worth: 1081.5969878247506
Step: 3520, Net Worth: 1081.5969878247506
Step: 3521, Net Worth: 1081.5969878247506
Step:

Step: 426, Net Worth: 984.1438915093431
Step: 427, Net Worth: 1014.3874487961776
Step: 428, Net Worth: 1017.2703868211379
Step: 429, Net Worth: 1017.6058512312637
Step: 430, Net Worth: 1028.5236631220948
Step: 431, Net Worth: 1028.5236631220948
Step: 432, Net Worth: 1053.5318802971085
Step: 433, Net Worth: 1053.5318802971085
Step: 434, Net Worth: 1055.061872579101
Step: 435, Net Worth: 1055.061872579101
Step: 436, Net Worth: 1055.061872579101
Step: 437, Net Worth: 1055.061872579101
Step: 438, Net Worth: 1055.061872579101
Step: 439, Net Worth: 1055.061872579101
Step: 440, Net Worth: 1043.7913126384774
Step: 441, Net Worth: 1043.7913126384774
Step: 442, Net Worth: 1043.7913126384774
Step: 443, Net Worth: 1044.4294785956895
Step: 444, Net Worth: 1044.4294785956895
Step: 445, Net Worth: 1044.4294785956895
Step: 446, Net Worth: 1044.4294785956895
Step: 447, Net Worth: 1047.3105868560026
Step: 448, Net Worth: 1045.915477435435
Step: 449, Net Worth: 1043.3728736243886
Step: 450, Net Worth: 10

...

Step: 1111, Net Worth: 1064.5643916659037
Step: 1112, Net Worth: 1064.5643916659037
Step: 1113, Net Worth: 1064.5643916659037
Step: 1114, Net Worth: 1064.5643916659037
Step: 1115, Net Worth: 1064.5643916659037
Step: 1116, Net Worth: 1070.2152940400385
Step: 1117, Net Worth: 1070.2152940400385
Step: 1118, Net Worth: 1070.2152940400385
Step: 1119, Net Worth: 1070.2152940400385
Step: 1120, Net Worth: 1070.2152940400385
Step: 1121, Net Worth: 1070.2152940400385
Step: 1122, Net Worth: 1070.2152940400385
Step: 1123, Net Worth: 1070.800454584281
Step: 1124, Net Worth: 1069.1629524769646
Step: 1125, Net Worth: 1069.1629524769646
Step: 1126, Net Worth: 1069.1629524769646
Step: 1127, Net Worth: 1061.249373576722
Step: 1128, Net Worth: 1061.249373576722
Step: 1129, Net Worth: 1055.9967341224353
Step: 1130, Net Worth: 1057.5142396943436
Step: 1131, Net Worth: 1057.9261842795702
Step: 1132, Net Worth: 1057.9261842795702
Step: 1133, Net Worth: 1057.9261842795702
Step: 1134, Net Worth: 1057.926184279

Step: 2175, Net Worth: 1038.7596407064157
Step: 2176, Net Worth: 1046.9360169041636
Step: 2177, Net Worth: 1053.493309218365
Step: 2178, Net Worth: 1057.528762241841
Step: 2179, Net Worth: 1057.528762241841
Step: 2180, Net Worth: 1057.528762241841
Step: 2181, Net Worth: 1068.3440664225834
Step: 2182, Net Worth: 1066.7514805151234
Step: 2183, Net Worth: 1069.6145645563188
Step: 2184, Net Worth: 1076.5190192046077
Step: 2185, Net Worth: 1073.6360869164396
Step: 2186, Net Worth: 1079.0290085068284
Step: 2187, Net Worth: 1061.0392685829836
Step: 2188, Net Worth: 1093.6325656992476
Step: 2189, Net Worth: 1086.5568462007445
Step: 2190, Net Worth: 1084.144307490282
Step: 2191, Net Worth: 1088.7366149550799
Step: 2192, Net Worth: 1088.7366149550799
Step: 2193, Net Worth: 1093.3039393029655
Step: 2194, Net Worth: 1093.3039393029655
Step: 2195, Net Worth: 1097.856128896187
Step: 2196, Net Worth: 1086.6790245540146
Step: 2197, Net Worth: 1086.6790245540146
Step: 2198, Net Worth: 1087.221282760074

Step: 2491, Net Worth: 973.0661771771681
Step: 2492, Net Worth: 979.1584164542624
Step: 2493, Net Worth: 979.1584164542624
Step: 2494, Net Worth: 986.2389478622152
Step: 2495, Net Worth: 986.2389478622152
Step: 2496, Net Worth: 986.2389478622152
Step: 2497, Net Worth: 986.2389478622152
Step: 2498, Net Worth: 987.3688948507589
Step: 2499, Net Worth: 987.3688948507589
Step: 2500, Net Worth: 987.3688948507589
Step: 2501, Net Worth: 987.3688948507589
Step: 2502, Net Worth: 987.3688948507589
Step: 2503, Net Worth: 984.9788675956103
Step: 2504, Net Worth: 984.9788675956103
Step: 2505, Net Worth: 984.9788675956103
Step: 2506, Net Worth: 984.9788675956103
Step: 2507, Net Worth: 984.9788675956103
Step: 2508, Net Worth: 973.9413375220695
Step: 2509, Net Worth: 981.9265264248083
Step: 2510, Net Worth: 981.2106778069721
Step: 2511, Net Worth: 970.2451304873431
Step: 2512, Net Worth: 970.2451304873431
Step: 2513, Net Worth: 968.21694331513
Step: 2514, Net Worth: 964.1316992833874
Step: 2515, Net Wo

Step: 1033, Net Worth: 1161.9507263658581
Step: 1034, Net Worth: 1178.7739178577867
Step: 1035, Net Worth: 1178.7739178577867
Step: 1036, Net Worth: 1178.7739178577867
Step: 1037, Net Worth: 1178.7739178577867
Step: 1038, Net Worth: 1190.7506648745098
Step: 1039, Net Worth: 1190.7506648745098
Step: 1040, Net Worth: 1190.7506648745098
Step: 1041, Net Worth: 1186.1470588375646
Step: 1042, Net Worth: 1186.1470588375646
Step: 1043, Net Worth: 1186.1470588375646
Step: 1044, Net Worth: 1186.1470588375646
Step: 1045, Net Worth: 1173.2320659367467
Step: 1046, Net Worth: 1173.2320659367467
Step: 1047, Net Worth: 1169.4464730654804
Step: 1048, Net Worth: 1166.4683856057186
Step: 1049, Net Worth: 1163.5405881701333
Step: 1050, Net Worth: 1163.5405881701333
Step: 1051, Net Worth: 1162.3104189419803
Step: 1052, Net Worth: 1150.7932609523737
Step: 1053, Net Worth: 1132.674262661441
Step: 1054, Net Worth: 1124.4190532026785
Step: 1055, Net Worth: 1124.887681091137
Step: 1056, Net Worth: 1124.88768109

Step: 4059, Net Worth: 1115.7460565125448
Step: 4060, Net Worth: 1103.0438470337
Step: 4061, Net Worth: 1106.0527178423717
Step: 4062, Net Worth: 1106.0527178423717
Step: 4063, Net Worth: 1106.0527178423717
Step: 4064, Net Worth: 1106.0527178423717
Step: 4065, Net Worth: 1106.0527178423717
Step: 4066, Net Worth: 1089.5115621428918
Step: 4067, Net Worth: 1081.730172715186
Step: 4068, Net Worth: 1089.8403710262694
Step: 4069, Net Worth: 1089.8403710262694
Step: 4070, Net Worth: 1089.8403710262694
Step: 4071, Net Worth: 1089.8403710262694
Step: 4072, Net Worth: 1089.8403710262694
Step: 4073, Net Worth: 1089.8403710262694
Step: 4074, Net Worth: 1089.8403710262694
Step: 4075, Net Worth: 1089.8403710262694
Step: 4076, Net Worth: 1089.8403710262694
Step: 4077, Net Worth: 1089.8403710262694
Step: 4078, Net Worth: 1089.8403710262694
Step: 4079, Net Worth: 1089.8403710262694
Step: 4080, Net Worth: 1089.8403710262694
Step: 4081, Net Worth: 1089.8403710262694
Step: 4082, Net Worth: 1066.5538415487